<a href="https://www.kaggle.com/code/abasiama/adversarial-attack-and-defense?scriptVersionId=209236588" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## **Title:  Robust Network Intrusion Detection System**

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier

Download data from kaggle

In [2]:
# Replace 'your_kaggle_api_key.json' with the actual path to your Kaggle API key file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d sweety18/cicids2017-full-dataset -f combine.csv

# Unzip the downloaded file
!unzip combine.csv.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/sweety18/cicids2017-full-dataset
License(s): unknown
 99%|████████████████████████████████████████▊| 171M/172M [00:00<00:00, 231MB/s]
100%|█████████████████████████████████████████| 172M/172M [00:00<00:00, 185MB/s]
Archive:  combine.csv.zip
  inflating: combine.csv             


In [3]:
pd.set_option("display.max_rows",None,"display.max_columns",None)

Import needed libraries

Read data into pandas dataframe called "df

In [4]:
df = pd.read_csv("/kaggle/working/combine.csv")

/tmp/ipykernel_17/476653546.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/working/combine.csv")


Check first five rows

In [5]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,33.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,55054,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,55055,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,46236,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,31.0,329.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,54863,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,32.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


Check the number of columns and rows

In [6]:
df.shape

(2214469, 79)

Inspect the datatypes of each of the variables

In [7]:
df.dtypes

 Destination Port                object
 Flow Duration                  float64
 Total Fwd Packets              float64
 Total Backward Packets         float64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    float64
 Flow Packets/s                 float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                   float64
 Flow IAT Min                   float64
Fwd IAT Total                   float64
 Fwd IAT Mean                   float64
 Fwd IAT Std                    float64
 Fwd IAT Max                    float64
 Fwd IAT Min                    float64


Unique values  wihin " Destination Port"

In [8]:
df[" Destination Port"].value_counts()

 Destination Port
53       688871
80       552191
443      372431
123       14988
22         7873
137        5315
389        4597
21         4076
88         3826
8080       2743
465        2718
139        2180
3268       1812
53         1616
445        1609
0          1239
135        1141
138        1118
49666       526
5353        484
444         361
1042        348
1041        344
1900        343
1           341
1026        341
1037        339
1124        313
8081        274
1434        272
42          269
5355        255
50636       254
50000       248
49400       246
50300       246
50002       243
50003       243
55056       242
49999       242
48080       242
49175       241
52822       241
52673       240
50500       240
55600       238
57294       238
51103       238
44176       238
54045       237
52848       237
50006       237
54328       237
56737       237
8000        236
51493       236
50001       236
50800       236
49176       235
49158       235
52869       235
55055 

hecking for the presence of mssing values

In [9]:
df.isnull().sum()

 Destination Port                  0
 Flow Duration                     1
 Total Fwd Packets                 1
 Total Backward Packets            1
Total Length of Fwd Packets        1
 Total Length of Bwd Packets       1
 Fwd Packet Length Max             1
 Fwd Packet Length Min             1
 Fwd Packet Length Mean            1
 Fwd Packet Length Std             1
Bwd Packet Length Max              1
 Bwd Packet Length Min             1
 Bwd Packet Length Mean            1
 Bwd Packet Length Std             1
Flow Bytes/s                    1138
 Flow Packets/s                    1
 Flow IAT Mean                     1
 Flow IAT Std                      1
 Flow IAT Max                      1
 Flow IAT Min                      1
Fwd IAT Total                      1
 Fwd IAT Mean                      1
 Fwd IAT Std                       1
 Fwd IAT Max                       1
 Fwd IAT Min                       1
Bwd IAT Total                      1
 Bwd IAT Mean                      1
 

Missing values by percentage

In [10]:
df.isnull().mean()

 Destination Port               0.000000e+00
 Flow Duration                  4.515755e-07
 Total Fwd Packets              4.515755e-07
 Total Backward Packets         4.515755e-07
Total Length of Fwd Packets     4.515755e-07
 Total Length of Bwd Packets    4.515755e-07
 Fwd Packet Length Max          4.515755e-07
 Fwd Packet Length Min          4.515755e-07
 Fwd Packet Length Mean         4.515755e-07
 Fwd Packet Length Std          4.515755e-07
Bwd Packet Length Max           4.515755e-07
 Bwd Packet Length Min          4.515755e-07
 Bwd Packet Length Mean         4.515755e-07
 Bwd Packet Length Std          4.515755e-07
Flow Bytes/s                    5.138929e-04
 Flow Packets/s                 4.515755e-07
 Flow IAT Mean                  4.515755e-07
 Flow IAT Std                   4.515755e-07
 Flow IAT Max                   4.515755e-07
 Flow IAT Min                   4.515755e-07
Fwd IAT Total                   4.515755e-07
 Fwd IAT Mean                   4.515755e-07
 Fwd IAT S

Check for imbalance of the Target variable

In [11]:
df[' Label'].value_counts()

 Label
BENIGN              1672837
DoS Hulk             231073
PortScan             158930
DDoS                 128027
DoS GoldenEye         10293
DoS slowloris          5796
DoS Slowhttptest       5499
Bot                    1966
Infiltration             36
Heartbleed               11
Name: count, dtype: int64

Convert the Target variable into binomial variable with two values: Benign and Attck. Attack is the combination of all attacks present within the dataset.

In [12]:
# List of attack labels to combine
attack_labels = ["DoS Hulk", "PortScan", "DDoS", "DoS GoldenEye", "DoS slowloris",
                 "DoS Slowhttptest", "Bot", "Infiltration", "Heartbleed"]

# Replace specified attack labels with "Attack"
df.loc[df[' Label'].isin(attack_labels), ' Label'] = "Attack"

In [13]:
# Calculate the percentage of each label
label_counts = df[' Label'].value_counts(normalize=True) * 100

# Print the percentages
label_counts

 Label
BENIGN    75.541259
Attack    24.458741
Name: proportion, dtype: float64

There is a clear case of imbalance in the dataset with 24.46% of the dataset representing Attack while the remaining 75.54% represent Normal network.The data will have to be balanced to avoid the model not learning much from the under-represented class. This will be done using udersampling method.

Handling Rare lables:

The  were rare labels in the "' Destination Port" variable. The top 5 were retained while the rest were represented as others.

In [14]:
# Group by 'Destination Port' and get the top 5 ports
top_ports = df[' Destination Port'].value_counts().nlargest(5).index


In [15]:
top_ports

Index([53, 80, 443, 123, 22], dtype='int64', name=' Destination Port')

In [16]:
# Create a new column 'Destination Port Grouped'
df['Destination Port Grouped'] = df[' Destination Port'].apply(lambda x: x if x in top_ports else 'Others')

In [17]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Destination Port Grouped
0,54865,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,33.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,Others
1,55054,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,Others
2,55055,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,Others
3,46236,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,31.0,329.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,Others
4,54863,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,32.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,Others


convert the process

In [18]:
df["Destination Port Grouped"].unique()

array(['Others', 53, 443, 80, 22, 123], dtype=object)

convert ' Destination Port' to string

In [19]:
df['Destination Port Grouped'] = df['Destination Port Grouped'].astype('string')

Convert ' Destination Port Grouped' to numerical vectors. This was done using Ordinal encoding as seen below:

In [20]:
# Create a mapping dictionary for ordinal encoding
port_mapping = {port: i for i, port in enumerate(df['Destination Port Grouped'].unique())}

# Apply ordinal encoding to the 'Destination Port Grouped' column
df['Destination Port Grouped Encoded'] = df['Destination Port Grouped'].map(port_mapping)

In [21]:
df['Destination Port Grouped Encoded'].value_counts()

Destination Port Grouped Encoded
1    688871
0    578115
3    552191
2    372431
5     14988
4      7873
Name: count, dtype: int64

In [22]:
df = df.dropna()

In [23]:
df[' Label'].value_counts()

 Label
BENIGN    1672649
Attack     540682
Name: count, dtype: int64

In [24]:
# Create a mapping dictionary for the target variable
label_mapping = {"BENIGN": 0, "Attack": 1}

# Apply the mapping to the 'Label' column
df[' Label'] = df[' Label'].map(label_mapping)

df[' Label'].value_counts()

 Label
0    1672649
1     540682
Name: count, dtype: int64

Under-Sampling to handle the imbalance in the target class

In [25]:
# Separate majority and minority classes
df_majority = df[df[' Label'] == 0]
df_minority = df[df[' Label'] == 1]

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts
df_downsampled[' Label'].value_counts()

 Label
0    540682
1    540682
Name: count, dtype: int64

Drop redundant variable

In [26]:
df = df.drop([' Destination Port','Destination Port Grouped'], axis=1)

drop null values

In [27]:
# df = df.dropna()

In [28]:
df.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Destination Port Grouped Encoded
0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,33.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,29.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,31.0,329.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0,0.0,32.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


confirm the removal of null values

In [29]:
df.isnull().sum()

 Flow Duration                      0
 Total Fwd Packets                  0
 Total Backward Packets             0
Total Length of Fwd Packets         0
 Total Length of Bwd Packets        0
 Fwd Packet Length Max              0
 Fwd Packet Length Min              0
 Fwd Packet Length Mean             0
 Fwd Packet Length Std              0
Bwd Packet Length Max               0
 Bwd Packet Length Min              0
 Bwd Packet Length Mean             0
 Bwd Packet Length Std              0
Flow Bytes/s                        0
 Flow Packets/s                     0
 Flow IAT Mean                      0
 Flow IAT Std                       0
 Flow IAT Max                       0
 Flow IAT Min                       0
Fwd IAT Total                       0
 Fwd IAT Mean                       0
 Fwd IAT Std                        0
 Fwd IAT Max                        0
 Fwd IAT Min                        0
Bwd IAT Total                       0
 Bwd IAT Mean                       0
 Bwd IAT Std

Remove  " infinity or a value too large for dtype('float64')" form dataset (df)

In [30]:
# Check for infinity and replace with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df.dropna(inplace=True)

Feature selection using selectKbest

In [31]:
X = df.drop(' Label', axis=1)  # Features
y = df[' Label']  # Target variable

# Apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_classif, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

# Concatenate dataframes
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  # Naming the dataframe columns

# Print 10 best features
print(featureScores.nlargest(50,'Score'))

                               Specs          Score
12             Bwd Packet Length Std  800364.842033
9              Bwd Packet Length Max  748401.171999
11            Bwd Packet Length Mean  735719.559748
53              Avg Bwd Segment Size  735719.559748
40                 Packet Length Std  645395.321212
38                 Max Packet Length  599194.059064
41            Packet Length Variance  562027.454946
39                Packet Length Mean  503661.024595
51               Average Packet Size  495877.086679
21                       Fwd IAT Std  487670.414937
75                          Idle Max  431131.447123
73                         Idle Mean  422471.822547
17                      Flow IAT Max  417140.613664
22                       Fwd IAT Max  417109.420234
76                          Idle Min  397997.714445
16                      Flow IAT Std  317558.210648
37                 Min Packet Length  265885.711973
77  Destination Port Grouped Encoded  260906.981160
10          

/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 32 55 56 57 58 59 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [32]:
df.columns

Index([' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SY

In [33]:
X = df[
    [
        " Bwd Packet Length Std",
        " Bwd Packet Length Mean",
        " Avg Bwd Segment Size",
        "Bwd Packet Length Max",
        " Packet Length Std",
        " Min Packet Length",
        " Max Packet Length",
        " Bwd Packet Length Min",
        " Packet Length Mean",
        " Average Packet Size",
        " Packet Length Variance",
        " Fwd IAT Std",
        " Idle Max",
        " Fwd IAT Max",
        " Flow IAT Max",
        "Idle Mean",
        " Idle Min",
        "Destination Port Grouped Encoded",
        " Flow IAT Std",
    ]
]

y = df[' Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Adjust test_size as needed

# Now you have X_train, X_test, y_train, and y_test for your model training and evaluation
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1769624, 19) (442406, 19) (1769624,) (442406,)


Scalling:

This is done to bring the dataset to the same magnitude to avoid one variable dorminating others.

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # Ensure the test set is scaled the same way

In [35]:
# Assuming y_test is already defined from the previous code
# Save y_test to a CSV file
y_test.to_csv('y_test.csv', index=False)

In [36]:
# Save X_test to a CSV file
pd.DataFrame(X_test).to_csv('X_test.csv', index=False)

Model Training and Evaluation - Buiding the IDS (Intrusion Detection System)


Decision Tree

In [37]:
# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42) # You can adjust hyperparameters here

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.997059262306569


In [38]:
import joblib

# Save the model to a file
joblib.dump(clf, 'decision_tree_model.joblib')

['decision_tree_model.joblib']

# **Adversarial Attacks**

# **FGSM**

In [39]:
epsilon = 0.1

# Function to generate adversarial examples using FGSM
def fgsm_attack(model, data, label, epsilon):
    data = np.array(data, dtype=np.float32)  # Ensure image data type
    # Find the predicted label for the input image
    initial_pred = model.predict(np.array([data]))[0]

    # If the initial prediction is correct, proceed with attack
    if initial_pred == label:
        # Calculate the loss (assuming a loss function that depends on model prediction and true label)
        # For simplicity, we'll just consider the prediction as the loss here (can be expanded)
        loss = initial_pred

        # Calculate the gradient of the loss with respect to the input image
        # Note: Decision Trees don't have a gradient in the typical sense, so this is a simplification
        # In practice, you'd approximate the gradient or use another method for adversarial examples with tree-based models.
        # Here, we'll simulate a gradient using a small random vector.
        gradient = np.random.uniform(low=-1, high=1, size=data.shape)

        # Calculate the perturbation
        perturbation = epsilon * np.sign(gradient)

        # Create the adversarial example
        adversarial_data = data + perturbation
        adversarial_data = np.clip(adversarial_data, 0, 1) # ensure values are in 0 to 1 range

        return adversarial_data

    return data  # Return original image if initial prediction is incorrect


# Generate adversarial examples for the entire test set
adversarial_examples = []
for i in range(len(X_test)):
  adversarial_ex = fgsm_attack(clf, X_test[i], y_test.iloc[i], epsilon)
  adversarial_examples.append(adversarial_ex)


# Evaluate the model on the adversarial examples
adversarial_predictions = clf.predict(np.array(adversarial_examples))

# Calculate accuracy on the adversarial examples
adversarial_accuracy = accuracy_score(y_test, adversarial_predictions)
print(f"Adversarial Accuracy: {adversarial_accuracy}")

# Compare the original and adversarial predictions
print("Original Predictions:", clf.predict(X_test)[:20]) # first 20
print("Adversarial Predictions:", adversarial_predictions[:20])

Adversarial Accuracy: 0.7518636727349991
Original Predictions: [0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0]
Adversarial Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [40]:
adversarial_examples[:20]

[array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.65308698, 0.        , 0.93043021, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.25401198, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.574968  , 0.67849151, 0.67849151, 1.        , 0.53148928,
        0.        , 0.93713973, 0.        , 0.81394759, 0.56368681,
        0.10437558, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.57564188, 0.        ]),
 array([0., 0., 0., 0., 1.

# **JSMA (Jacobian-based Saliency Map Attack)**

In [41]:
# Placeholder for Jacobian calculation (replace with actual Jacobian computation)
def calculate_jacobian(model, image):
    # This is a placeholder.  Replace with actual Jacobian calculation
    # for your decision tree model.  Decision trees don't have gradients
    # in the traditional sense. You'll likely need to approximate the Jacobian
    # or use a different method for adversarial examples with tree-based models.
    return np.random.rand(*image.shape)


def jsma_attack(model, image, label, theta, gamma, max_iterations):
    """
    Performs the Jacobian-based Saliency Map Attack (JSMA).
    """

    image = np.array(image, dtype=np.float32)
    modified_image = image.copy()
    current_prediction = model.predict(np.array([modified_image]))[0]

    for _ in range(max_iterations):
        jacobian = calculate_jacobian(model, modified_image)

        # Placeholder for saliency map calculation
        saliency_map = np.abs(jacobian)  # Replace with your actual saliency calculation

        # Placeholder for finding pixels to modify
        pixel_to_increase = np.unravel_index(np.argmax(saliency_map), saliency_map.shape)
        pixel_to_decrease = np.unravel_index(np.argmin(saliency_map), saliency_map.shape)

        # Update pixels (replace with your actual update logic)
        modified_image[pixel_to_increase] = np.clip(modified_image[pixel_to_increase] + theta, 0, 1)
        modified_image[pixel_to_decrease] = np.clip(modified_image[pixel_to_decrease] - gamma, 0, 1)


        new_prediction = model.predict(np.array([modified_image]))[0]

        # Check if the prediction has changed
        if new_prediction != current_prediction:
            break

        current_prediction = new_prediction #update current_prediction

    return modified_image



# Example usage (assuming you have X_test, y_test, and clf loaded)
theta = 0.1  # Adjust these parameters
gamma = 0.1
max_iterations = 10

jsma_adversarial_examples = []
for i in range(len(X_test)):
    jsma_ex = jsma_attack(clf, X_test[i], y_test.iloc[i], theta, gamma, max_iterations)
    jsma_adversarial_examples.append(jsma_ex)

# Evaluate the model on the JSMA adversarial examples
jsma_adversarial_predictions = clf.predict(np.array(jsma_adversarial_examples))

# Calculate accuracy on the JSMA adversarial examples
jsma_adversarial_accuracy = accuracy_score(y_test, jsma_adversarial_predictions)
print(f"JSMA Adversarial Accuracy: {jsma_adversarial_accuracy}")

print("JSMA Adversarial Predictions:", jsma_adversarial_predictions[:20])

JSMA Adversarial Accuracy: 0.6596271298309697
JSMA Adversarial Predictions: [0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0]


# **BIM  Attack**

In [42]:
def bim_attack(model, image, label, epsilon, alpha, iterations):
    """
    Performs the Basic Iterative Method (BIM) attack.
    """
    image = np.array(image, dtype=np.float32)
    perturbed_image = image.copy()

    for _ in range(iterations):
        # Calculate the gradient (placeholder, replace with actual gradient computation)
        # Decision trees don't have gradients, so this is an approximation
        gradient = np.random.uniform(low=-1, high=1, size=image.shape)

        # Update the perturbed image
        perturbed_image = perturbed_image + alpha * np.sign(gradient)

        # Project back to the feasible region (clip values)
        perturbed_image = np.clip(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = np.clip(perturbed_image, 0, 1)  # Ensure pixel values stay within valid range


    return perturbed_image


# Example usage
epsilon = 0.1
alpha = 0.01
iterations = 10

bim_adversarial_examples = []
for i in range(len(X_test)):
    bim_ex = bim_attack(clf, X_test[i], y_test.iloc[i], epsilon, alpha, iterations)
    bim_adversarial_examples.append(bim_ex)


# Evaluate the model on the BIM adversarial examples
bim_adversarial_predictions = clf.predict(np.array(bim_adversarial_examples))

# Calculate accuracy on the BIM adversarial examples
bim_adversarial_accuracy = accuracy_score(y_test, bim_adversarial_predictions)
print(f"BIM Adversarial Accuracy: {bim_adversarial_accuracy}")

print("BIM Adversarial Predictions:", bim_adversarial_predictions[:20])

BIM Adversarial Accuracy: 0.7572162222031347
BIM Adversarial Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# **Adversarial Defense**

Defense Against FGSM ATTACK

In [43]:
from sklearn.metrics import accuracy_score

# Combine original and adversarial examples for adversarial training
X_train_adv = np.vstack((X_train, adversarial_examples))
y_train_adv = np.hstack((y_train, y_test))  # Duplicate labels for adversarial examples

# Define AdaBoost classifier
clf_adv = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),  # Decision stump as the base estimator
    n_estimators=50,  # Number of boosting rounds
    random_state=42
)

# Retrain AdaBoost with adversarial examples
clf_adv.fit(X_train_adv, y_train_adv)

# Evaluate on the clean test set
clean_test_accuracy = accuracy_score(y_test, clf_adv.predict(X_test))
print(f"Clean Test Accuracy After Adversarial Training: {clean_test_accuracy}")

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Clean Test Accuracy After Adversarial Training: 0.987805319096034


## Defense Against Jacobian-based Map Attack (JSMA)

In [44]:
from sklearn.metrics import accuracy_score

# Combine original and adversarial examples for adversarial training
X_train_adv = np.vstack((X_train, jsma_adversarial_examples))
y_train_adv = np.hstack((y_train, y_test))  # Duplicate labels for adversarial examples

# Define AdaBoost classifier
clf_adv = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),  # Decision stump as the base estimator
    n_estimators=50,  # Number of boosting rounds
    random_state=42
)

# Retrain AdaBoost with adversarial examples
clf_adv.fit(X_train_adv, y_train_adv)

# Evaluate on the clean test set
clean_test_accuracy = accuracy_score(y_test, clf_adv.predict(X_test))
print(f"Clean Test Accuracy After Adversarial Training: {clean_test_accuracy}")

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Clean Test Accuracy After Adversarial Training: 0.986822059375325


## Defense Against BIM Attack

In [45]:
from sklearn.metrics import accuracy_score

# Combine original and adversarial examples for adversarial training
X_train_adv = np.vstack((X_train, bim_adversarial_examples))
y_train_adv = np.hstack((y_train, y_test))  # Duplicate labels for adversarial examples

# Define AdaBoost classifier
clf_adv = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),  # Decision stump as the base estimator
    n_estimators=50,  # Number of boosting rounds
    random_state=42
)

# Retrain AdaBoost with adversarial examples
clf_adv.fit(X_train_adv, y_train_adv)

# Evaluate on the clean test set
clean_test_accuracy = accuracy_score(y_test, clf_adv.predict(X_test))
print(f"Clean Test Accuracy After Adversarial Training: {clean_test_accuracy}")

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Clean Test Accuracy After Adversarial Training: 0.9861439492231118
